## Import Modules

In [1]:
import torch
import torch.nn.functional as F

import numpy as np
import json
import torchvision.transforms as transforms

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import skimage.transform
import argparse

from scipy.misc import imread, imresize
from PIL import Image

from utils.url import is_absolute_path, read_image_from_url

import warnings
warnings.filterwarnings('ignore')

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Functions

In [3]:
def caption_image_beam_search(encoder, encoder_tagger, decoder, image_path, word_map, beam_size=3):
    r"""Reads an image and captions it with beam search.

    Arguments
        encoder: encoder model
        decoder: decoder model
        image_path: path to image
        word_map: word map
        beam_size: number of sequences to consider at each decode-step
    Return
        String : caption
        Float  : weights for visualization
    """

    k = beam_size
    vocab_size = len(word_map)

    # Read image and process
    img = imread(image_path)
    if len(img.shape) == 2:
        img = img[:, :, np.newaxis]
        img = np.concatenate([img, img, img], axis=2)
    img = imresize(img, (256, 256))
    img = img.transpose(2, 0, 1)
    img = img / 255.
    img = torch.FloatTensor(img)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([normalize])
    image = transform(img).to(device)  # (3, 256, 256)

    # Encode
    image = image.unsqueeze(0)  # (1, 3, 256, 256)
    # (1, enc_image_size, enc_image_size, encoder_dim)
    encoder_out = encoder(image)
    enc_image_size = encoder_out.size(1)
    encoder_dim = encoder_out.size(3)

    tag_out = encoder_tagger(image)
    tag_size = tag_out.size(1)

    # Flatten encoding
    # (1, num_pixels, encoder_dim)
    encoder_out = encoder_out.view(1, -1, encoder_dim)
    num_pixels = encoder_out.size(1)

    # We'll treat the problem as having a batch size of k
    # (k, num_pixels, encoder_dim)
    encoder_out = encoder_out.expand(k, num_pixels, encoder_dim)

    temp_tag_out = tag_out.expand(k, tag_size)

    # Tensor to store top k previous words at each step; now they're just <start>
    k_prev_words = torch.LongTensor(
        [[word_map['<start>']]] * k).to(device)  # (k, 1)

    # Tensor to store top k sequences; now they're just <start>
    seqs = k_prev_words  # (k, 1)

    # Tensor to store top k sequences' scores; now they're just 0
    top_k_scores = torch.zeros(k, 1).to(device)  # (k, 1)

    # Lists to store completed sequences and scores
    complete_seqs = list()
    complete_seqs_scores = list()

    # Start decoding
    step = 1
    h, c = decoder.init_hidden_state(encoder_out)

    # s is a number less than or equal to k, because sequences are removed from this process once they hit <end>
    while True:

        embeddings = decoder.embedding(
            k_prev_words).squeeze(1)  # (s, embed_dim)

        h, c = decoder.decode_step(embeddings, temp_tag_out, (h, c))  # (s, decoder_dim)

        scores = decoder.fc(h)  # (s, vocab_size)
        scores = F.log_softmax(scores, dim=1)

        # Add
        scores = top_k_scores.expand_as(scores) + scores  # (s, vocab_size)

        # For the first step, all k points will have the same scores (since same k previous words, h, c)
        if step == 1:
            top_k_scores, top_k_words = scores[0].topk(k, 0, True, True)  # (s)
        else:
            # Unroll and find top scores, and their unrolled indices
            # (s)
            top_k_scores, top_k_words = scores.view(-1).topk(k, 0, True, True)

        # Convert unrolled indices to actual indices of scores
        prev_word_inds = top_k_words / vocab_size  # (s)
        next_word_inds = top_k_words % vocab_size  # (s)

        # Add new words to sequences
        seqs = torch.cat(
            [seqs[prev_word_inds], next_word_inds.unsqueeze(1)], dim=1)  # (s, step+1)

        # Which sequences are incomplete (didn't reach <end>)?
        incomplete_inds = [ind for ind, next_word in enumerate(next_word_inds) if
                           next_word != word_map['<end>']]
        complete_inds = list(
            set(range(len(next_word_inds))) - set(incomplete_inds))

        # Set aside complete sequences
        if len(complete_inds) > 0:
            complete_seqs.extend(seqs[complete_inds].tolist())
            complete_seqs_scores.extend(top_k_scores[complete_inds])
        k -= len(complete_inds)  # reduce beam length accordingly

        # Proceed with incomplete sequences
        if k == 0:
            break

        seqs = seqs[incomplete_inds]
        h = h[prev_word_inds[incomplete_inds]]
        c = c[prev_word_inds[incomplete_inds]]
        encoder_out = encoder_out[prev_word_inds[incomplete_inds]]
        temp_tag_out = temp_tag_out[prev_word_inds[incomplete_inds]]
        top_k_scores = top_k_scores[incomplete_inds].unsqueeze(1)
        k_prev_words = next_word_inds[incomplete_inds].unsqueeze(1)

        # Break if things have been going on too long
        if step > 50:
            break
        step += 1

    i = complete_seqs_scores.index(max(complete_seqs_scores))
    seq = complete_seqs[i]

    return seq, tag_out

In [4]:
def visualize_att(image_path, seq, alphas, rev_word_map, smooth=True):
    r"""Visualizes caption with weights at every word.

    Adapted from paper authors' repo: https://github.com/kelvinxu/arctic-captions/blob/master/alpha_visualization.ipynb

    Arguments
        image_path: path to image that has been captioned
        seq: caption
        alphas: weights
        rev_word_map: reverse word mapping, i.e. ix2word
        smooth: smooth weights?
    """
    plt.figure(1, figsize=(24, 9))
    
    image = Image.open(image_path)
    image = image.resize([14 * 24, 14 * 24], Image.LANCZOS)

    words = [rev_word_map[ind] for ind in seq]

    for t in range(len(words)):
        if t > 50:
            break
        plt.subplot(np.ceil(len(words) / 5.), 5, t + 1)

        plt.text(0, 1, '%s' % (words[t]), color='black',
                 backgroundcolor='white', fontsize=12)
        plt.imshow(image)
        current_alpha = alphas[t, :]
        if smooth:
            alpha = skimage.transform.pyramid_expand(
                current_alpha.numpy(), upscale=24, sigma=8)
        else:
            alpha = skimage.transform.resize(
                current_alpha.numpy(), [14 * 24, 14 * 24])
        if t == 0:
            plt.imshow(alpha, alpha=0)
        else:
            plt.imshow(alpha, alpha=0.8)
        plt.set_cmap(cm.Greys_r)
        plt.axis('off')
    plt.show()

### Load Model

In [5]:
caption_checkpoint_path = './pretrained/BEST_checkpoint_scn_flickr10k_5_cap_per_img_5_min_word_freq.pth.tar'
tagger_checkpoint_path = './pretrained/BEST_checkpoint_tagger_flickr10k_5_cap_per_img_5_min_word_freq.pth.tar'
word_map_path = './scn_data/WORDMAP_flickr10k_5_cap_per_img_5_min_word_freq.json'
tag_map_path = './scn_data/TAGMAP_flickr10k_5_cap_per_img_5_min_word_freq.json'

#### Tagger Model

In [6]:
%%capture
tagger_checkpoint = torch.load(
        tagger_checkpoint_path, map_location=lambda storage, loc: storage)
encoder_tagger = tagger_checkpoint['encoder']
encoder_tagger = encoder_tagger.to(device)
encoder_tagger.eval()

#### Caption Model

In [7]:
caption_checkpoint = torch.load(caption_checkpoint_path, map_location=lambda storage, loc: storage)

FileNotFoundError: [Errno 2] No such file or directory: './pretrained/BEST_checkpoint_scn_flickr10k_5_cap_per_img_5_min_word_freq.pth.tar'

In [8]:
%%capture
decoder = caption_checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()

NameError: name 'caption_checkpoint' is not defined

In [ ]:
%%capture
encoder = caption_checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval()

### Load Word and Tag Vocab

In [ ]:
with open(word_map_path, 'r') as j:
    word_map = json.load(j)
rev_word_map = {v: k for k, v in word_map.items()}  # ix2word

In [ ]:
with open(tag_map_path, 'r') as j:
    tag_map = json.load(j)
rev_tag_map = {v: k for k, v in tag_map.items()}

### Variables

In [ ]:
file_img_path = 'http://farm4.staticflickr.com/3784/9364313863_612ce52cb0_z.jpg'
tag_out_count = 20
beam_size = 5
smooth = True

In [ ]:
if is_absolute_path(file_img_path):
    file_img = read_image_from_url(file_img_path)

<br />

### Show Image

In [ ]:
fig, ax = plt.subplots(figsize=(18,9))
img = imread(file_img)
plt.axis('off')
ax.imshow(img)

<br />

### Decode Caption!

In [ ]:
# Encode, decode with attention and beam search
seq, alphas, tags = caption_image_beam_search(encoder, encoder_tagger, decoder, file_img, word_map, beam_size)
alphas = torch.FloatTensor(alphas)

<br />

### Output Tags

In [ ]:
tags = np.asarray(tags.flatten().tolist())
tag_index = np.argsort(tags)[-tag_out_count:]

In [ ]:
for idx in tag_index:
    print('{} {}'.format(rev_tag_map[idx], tags[idx]))

<br />

### Get Caption!

In [ ]:
words = [rev_word_map[ind] for ind in seq]

In [ ]:
words